# PM2.5 concentration spatial distribution estimation in China 
## Abstract  

China, as the largest developing country in the world, has been facing increasingly severe air quality problems accompanying the continuous advancement of industrialization and urbanization. According to the “2017 China Environmental Situation Bulletin” released by the Ministry of Ecology and Environment, among the 338 cities at the prefecture level and above, 239 cities have exceeded the national ambient air quality standards, with a proportion of more than 70%. Air quality problems have seriously affected people's daily travel and physical health, restricted sustainable economic development, and become a hot issue for public and government attention. PM2.5 is a respirable particle with a diameter of no more than 2.5 micrometers in the aerodynamic field, and is one of the main indicators for evaluating air quality. Comprehensive understanding of the spatial distribution of PM2.5 concentration, representing the spatial process and environmental behavior of atmospheric pollution, is of great significance and guidance value for supporting atmospheric pollution monitoring and early warning, comprehensive treatment, protecting human health and social sustainable development. By the end of 2017, the China National Environmental Monitoring Center had established over 400 ground-level air quality monitoring stations, and released hourly air quality monitoring data including PM2.5, providing high-precision and reliable real-time monitoring results. However, due to the uneven spatial distribution and low coverage of ground monitoring stations, it is difficult for existing studies to effectively analyze and deeply mine the spatiotemporal data of their monitoring data. Unlike ground monitoring, remote sensing observation based on satellites can obtain high-coverage atmospheric environment spatial datasets, such as atmospheric aerosol optical depth (AOD). Numerous studies have shown that there is a strong correlation between AOD and PM2.5 concentration. Research on PM2.5 concentration Spatial regression relationships with factors such as AOD retrieved from remote sensing inversion can provide effective solutions for obtaining the spatial distribution of PM2.5 concentration in the entire study area. Methodology Based on GWR geographical weighted regression thinking, Wu SenSen combines OLR with neural network model to propose a Geographically Neural Network Weighted Regression (GNNWR) model. By utilizing the learning ability of neural networks, this model can handle the spatial heterogeneity and complex nonlinear characteristics of regression relationships, which has better fitness accuracy and prediction performance compared to models such as OLR and GWR. The purpose of this case is to establish a PM2.5 concentration spatial estimation model based on GNNWR to achieve accurate fitting of spatial heterogeneity and nonlinear characteristics in PM2.5 regression relationships, and then obtain high-precision and high-reasonability PM2.5 concentration spatial distribution in China.  

## Data description  

Many studies have shown that integrating meteorological conditions such as temperature, precipitation, wind speed, wind direction, and surface elevation factors can further improve the accuracy of PM2.5 spatial estimation. In this case, in addition to selecting AOD data as an auxiliary factor, temperature (TEMP), precipitation (TP), wind speed (WS), wind direction (WD), and surface elevation (DEM) factors are added as input variables for the model. The research time scale is the average of 2017 year scale:
(1) PM2.5 monitoring site data. The hourly PM2.5 concentration observation values from January 1, 2017 to December 31, 2017 were obtained from the China Environmental Monitoring Station. PM2.5 concentrations were measured using cone-shaped element oscillation trace or beta attenuation methods following national standard GB3095-2012. PM2.5 data were averaged for one year's time scale
(2) Aerosol data. Aerosol data are obtained from the LAADS website including both Terra and Aqua dark pixel inversion products with a resolution of 3 km (MOD04_3K and MYD04_3k), as well as deep blue algorithm inversion products with a resolution of 10 km (MOD04_L2 and MYD04_L2). In this article, the 3 km resolution AOD products are the main data source for PM2.5 estimation. When there is a missing value in the 3 km resolution data, a resolution-matching product will be used as much as possible using a 10 km resolution product for resampling substitution to ensure the reliability of AOD data.
(3) DEM data. DEM data are obtained from the ETO-PO1 global surface elevation model of NOAA with a resolution of 1 arc minute
(4) Meteorological data including temperature, precipitation, wind speed, and wind direction are obtained from the ERA5 global climate reanalysis modele provided by ECMWF with hourly gridded data at a resolution of 0.5 degrees.   

## Model Introduction  

Based on the geographical weighted idea similar to GWR, the GNNWR model believes that the spatial heterogeneity of the regression relationship can be regarded as the varying levels of spatial nonstationarity at different locations that affect the "OLR regression relationship". Therefore, in the spatial estimation experiment of PM2.5 concentration in this case, the model structure of GNNWR is defined as follows:  
 
![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694059648746_1.png)  

In this equation, (u_i, v_i) are the spatial coordinates of the i-th sample point, and β = (β_0, β_1, ..., β_6) are the regression coefficients of the OLR model, reflecting the average level of the PM2.5 regression relationship for the entire region. The estimation matrix of OLR coefficients is represented as follows:

![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694059665465_2.png)  

of which: 

![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694059673642_3.png)  


<br/><br/>  


![Image Name](https://mydde.deep-time.org/s3/static-files/upload/upload/1694003342595_1.png)  
The spatial estimation model for PM2.5 concentration based on GNNWR

## Main Content  
1. Model Training 
2. Result Storage, Loading, and Visualization
3. Estimation 

# Part A：Preparation

## Import Necessary Packages

In [1]:
from gnnwr import models,datasets
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap, MarkerCluster
import torch.nn as nn
from sklearn.metrics import r2_score as r2
import matplotlib.pyplot as plt

# Part B：Model Training

## Step 1：Import Training Data

In [2]:
data = pd.read_csv(u'../data/pm25_data.csv')
data.head(5)

,监测点编码,监测点名称,城市,经度,纬度,date,PM2_5,row_index,col_index,proj_x,...,t2m,sp,tp,blh,e,r,u10,v10,aod_sat,ndvi
0,1001A,万寿西宫,北京,116.366,39.8673,20170601,54.733894,2201,6867,1650847.552,...,284.561066,100809.27340,0.001006,134.995636,-0.000007,46.315975,0.425366,0.170262,0.870967,2401
1,1002A,定陵,北京,116.170,40.2865,20170601,48.080737,2134,6835,1625003.973,...,282.907684,97125.08594,0.001044,157.775970,-0.000006,53.605503,0.211734,-0.676848,0.712080,5255
2,1003A,东四,北京,116.434,39.9522,20170601,54.898592,2188,6877,1653776.710,...,284.492249,100830.96880,0.001002,129.971298,-0.000007,45.537464,0.266666,0.069172,0.875811,2609
3,1004A,天坛,北京,116.434,39.8745,20170601,52.266382,2200,6877,1655828.045,...,284.636200,100936.80470,0.001010,138.793961,-0.000007,45.387913,0.299403,0.227950,0.869679,2420
4,1005A,农展馆,北京,116.473,39.9716,20170601,53.189076,2185,6884,1656224.681,...,284.506561,100880.17970,0.001019,130.520599,-0.000007,44.790119,0.169121,0.079546,0.873232,3296


### View the distribution of training data

In [3]:
from demo_utils import marker_map

In [4]:
lon_center,lat_center = data['经度'].mean(),data['纬度'].mean()
locations = list(zip(data['纬度'],data['经度']))
markers = [{'location':locations,'color':'blue','desc':'data'}]
map = marker_map(markers,[lat_center,lon_center],4)
# tiles= 'https://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=en&size=1&scl=1&style=7'
# map = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = tiles,attr="高德")
# mc = MarkerCluster(locations=locations)
# mc.add_to(map)
map

## Step 2：Partition Datasets

In [5]:
train_dataset, val_dataset, test_dataset = datasets.init_dataset(data=data,
                                                        test_ratio=0.15,
                                                        valid_ratio=0.15,
                                                        x_column=['dem', 'w10','d10','t2m','aod_sat','tp'],
                                                        y_column=['PM2_5'],
                                                        spatial_column=['经度','纬度'],
                                                        sample_seed=23,
                                                        batch_size=64)

x_min:[-5.0000000e+00  4.1591436e-02  3.9565850e-02  2.6959613e+02
  5.6254357e-02  3.8816700e-05];  x_max:[4.52000000e+03 3.20341086e+00 3.59605225e+02 2.97242950e+02
 1.06999075e+00 4.07377200e-03]
y_min:[3.85633803];  y_max:[133.8005618]


### Checking the partitioning of the datasets

In [6]:
markers_1 = [{'location':list(zip(train_dataset.dataframe['纬度'],train_dataset.dataframe['经度'])),'color':'blue'},
             {'location':list(zip(val_dataset.dataframe['纬度'],val_dataset.dataframe['经度'])),'color':'green'},
             {'location':list(zip(test_dataset.dataframe['纬度'],test_dataset.dataframe['经度'])),'color':'red'}
            ]
map_1 = marker_map(markers_1,[lat_center,lon_center],4)
# train_dataset.dataframe.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],icon=folium.Icon(color='red'),popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map_1),axis=1)
# val_dataset.dataframe.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],icon=folium.Icon(color='green'),popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map_1),axis=1)
# test_dataset.dataframe.apply(lambda x:folium.Marker(location=[x['纬度'],x['经度']],icon=folium.Icon(color='blue'),popup=x['监测点名称']+'\n PM2.5: '+str(x['PM2_5'])).add_to(map_1),axis=1)

map_1

## Step 3：Initialize GNNWR Model

In [7]:
gnnwr = models.GNNWR(train_dataset = train_dataset,
                     valid_dataset = val_dataset, 
                     test_dataset = test_dataset,
                     dense_layers = [512, 256, 128],
                     start_lr = 0.2,
                     optimizer = "Adadelta",
                     activate_func = nn.PReLU(init=0.1),
                     model_name = "GNNWR_PM25",
                     model_save_path = "./demo_result/gnnwr_models",
                     log_path = "./demo_result/gnnwr_logs",
                     write_path = "./demo_result/gnnwr_runs"
                     )

## Step 4：Model Training

In [8]:
gnnwr.run(max_epoch = 200,print_frequency = 10)

  5%|▌         | 10/200 [00:04<01:22,  2.31it/s]


Epoch:  10
learning rate:  0.1962279001393096
Train Loss:  300.9492243914009
Train R2: -0.49429
Train RMSE: 17.34789
Train AIC: 8816.59598
Train AICc: 9880.83398
Valid Loss:  134.1769256591797
Valid R2: 0.29128 

Best R2: 0.53554 



 10%|█         | 20/200 [00:09<01:16,  2.35it/s]


Epoch:  20
learning rate:  0.18357265455608338
Train Loss:  204.82871451138746
Train R2: -0.01703
Train RMSE: 14.31184
Train AIC: 8512.92033
Train AICc: 9098.59277
Valid Loss:  113.14983367919922
Valid R2: 0.40235 

Best R2: 0.61569 



 15%|█▌        | 30/200 [00:13<01:11,  2.38it/s]

...


Epoch:  180
learning rate:  0.16930321711985277
Train Loss:  55.333547939124244
Train R2: 0.72525
Train RMSE: 7.43865
Train AIC: 8086.15489
Train AICc: 8136.54004
Valid Loss:  52.7077751159668
Valid R2: 0.72160 

Best R2: 0.74894 



 95%|█████████▌| 190/200 [01:22<00:04,  2.49it/s]


Epoch:  190
learning rate:  0.16164136312173352
Train Loss:  54.75441473751762
Train R2: 0.72813
Train RMSE: 7.39962
Train AIC: 8040.94469
Train AICc: 8078.79541
Valid Loss:  53.10469436645508
Valid R2: 0.71950 

Best R2: 0.74894 



100%|██████████| 200/200 [01:26<00:00,  2.31it/s]


Epoch:  200
learning rate:  0.1533589344962853
Train Loss:  54.46639216450229
Train R2: 0.72956
Train RMSE: 7.38014
Train AIC: 8058.65703
Train AICc: 8075.84033
Valid Loss:  52.44244384765625
Valid R2: 0.72300 

Best R2: 0.74894 

Best_r2: 0.7489443535163812


## Step 5：模型评价与结果分析

### 输出模型训练结果

In [9]:
gnnwr.result()


Test Loss:  59.99171829223633  Test R2:  0.722547501956343
--------------------Result Table--------------------

Model Name:           | GNNWR_PM25
Model Structure:      |
 SWNN(
  (activate_func): PReLU(num_parameters=1)
  (fc): Sequential(
    (swnn_full0): Linear(in_features=1017, out_features=512, bias=True)
    (swnn_batc0): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (swnn_acti0): PReLU(num_parameters=1)
    (swnn_drop0): Dropout(p=0.2, inplace=False)
    (swnn_full1): Linear(in_features=512, out_features=256, bias=True)
    (swnn_batc1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (swnn_acti1): PReLU(num_parameters=1)
    (swnn_drop1): Dropout(p=0.2, inplace=False)
    (swnn_full2): Linear(in_features=256, out_features=128, bias=True)
    (swnn_batc2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (swnn_acti2): PReLU(num_parameters=1)
    (swnn_drop2): Dropout

### Save the training result

In [10]:
gnnwr.reg_result('./demo_result/GNNWR_PM25_Result.csv')

,weight_dem,weight_w10,weight_d10,weight_t2m,weight_aod_sat,weight_tp,bias,Pred_PM2_5,id
0,-21.973896,-0.469382,-0.456472,45.690353,26.208168,-31.604027,29.272074,63.026634,400
1,-20.059723,-2.788708,-0.556628,29.267454,-7.930019,-28.678818,55.074280,61.174675,271
2,-11.090857,12.392816,-0.189018,-28.680275,77.995834,6.655582,-2.003451,31.145264,571
3,-7.934442,8.430605,-0.158527,2.637909,59.752037,-17.250937,9.904790,47.819176,618
4,-11.397975,8.106547,-0.246221,30.876463,13.238321,-12.304317,14.221577,37.470226,1158
...,...,...,...,...,...,...,...,...,...
1403,-18.758150,-5.743988,-0.551186,28.928928,-5.958023,-21.266756,54.789608,64.608917,637
1404,-9.070152,2.934910,-0.313578,23.026178,8.741288,-2.717492,15.379046,39.127567,711
1405,-12.554069,7.566235,-0.282979,13.902503,35.581516,-19.120310,22.968082,50.942772,966
1406,-9.969007,-3.075313,-0.121528,32.761562,37.500797,-2.668689,11.105102,33.367908,80


### Drawing the heat map for the distribution of weights of each variables 

In [11]:
result_data = gnnwr.getWeights()


Warning: The input write file path is not set, and the result is output to the default path.

#### DEM

In [12]:

hmap_dem = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_dem = [[row['纬度'],row['经度'],row['weight_dem']]for index,row in result_data.iterrows()]
hmap_dem.add_child(HeatMap(weight_dem))
hmap_dem

NameError: name 'result_data' is not defined

#### AOD

In [ ]:
hmap_aod = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_aod = [[row['纬度'],row['经度'],row['weight_aod_sat']]for index,row in result_data.iterrows()]
hmap_aod.add_child(HeatMap(weight_aod))
hmap_aod

### Precipitation

In [ ]:
hmap_tp = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_tp = [[row['纬度'],row['经度'],row['weight_tp']]for index,row in result_data.iterrows()]
hmap_tp.add_child(HeatMap(weight_tp))
hmap_tp

#### Temperature

In [ ]:
hmap_t2m = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_t2m = [[row['纬度'],row['经度'],row['weight_t2m']]for index,row in result_data.iterrows()]
hmap_t2m.add_child(HeatMap(weight_t2m))
hmap_t2m

#### Wind Speed

In [ ]:
hmap_w10 = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_w10 = [[row['纬度'],row['经度'],row['weight_w10']]for index,row in result_data.iterrows()]
hmap_w10.add_child(HeatMap(weight_w10))
hmap_w10

#### Wind Direction

In [ ]:
hmap_d10 = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_d10 = [[row['纬度'],row['经度'],row['weight_d10']]for index,row in result_data.iterrows()]
hmap_d10.add_child(HeatMap(weight_d10))
hmap_d10

#### Bias

In [ ]:
hmap_bias = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_bias = [[row['纬度'],row['经度'],row['bias']]for index,row in result_data.iterrows()]
hmap_bias.add_child(HeatMap(weight_bias))
hmap_bias

# Part C：Saving and Loading

## Step 1：Saving Datasets

In [ ]:
# make sure dir is not exist
train_dataset.save('./demo_result/gnnwr_datasets/train_dataset')
val_dataset.save('./demo_result/gnnwr_datasets/val_dataset')
test_dataset.save('./demo_result/gnnwr_datasets/test_dataset')

## Step 2：Loading Datasets

In [ ]:
train_dataset_load = datasets.load_dataset('./demo_result/gnnwr_datasets/train_dataset/')
val_dataset_load = datasets.load_dataset('./demo_result/gnnwr_datasets/val_dataset/')
test_dataset_load = datasets.load_dataset('./demo_result/gnnwr_datasets/test_dataset/')

## Step 3：Loading Model

### Initialize GNNWR Model

In [ ]:
gnnwr_load = models.GNNWR(train_dataset = train_dataset_load,
                     valid_dataset = val_dataset_load, 
                     test_dataset = test_dataset_load,
                     dense_layers = [512, 256, 128],
                     start_lr = 0.2,
                     optimizer = "Adadelta",
                     activate_func = nn.PReLU(init=0.1),
                     model_name = "GNNWR_PM25",
                     model_save_path = "./demo_result/gnnwr_models",
                     log_path = "./demo_result/gnnwr_logs",
                     write_path = "./demo_result/gnnwr_runs"
                     )

### Loading Parameters

In [ ]:
gnnwr_load.load_model('./demo_result/gnnwr_models/GNNWR_PM25.pkl')
gnnwr_load.result()

# Part D：Estimation

## Step 1：Import Estimation Data

In [ ]:
pred_data = pd.read_csv(u'../data/pm25_predict_data.csv')

## Step 2：Initialize Estimation Dataset

In [ ]:
pred_dataset = datasets.init_predict_dataset(data = pred_data,train_dataset = train_dataset,x_column=['dem', 'w10','d10','t2m','aod_sat','tp'],spatial_column=['经度','纬度'])

## Step 3：Estimate

In [ ]:
res = gnnwr.predict(pred_dataset)
res.head(5)

In [ ]:
hmap_pred = folium.Map(location=[lat_center,lon_center],zoom_start=4,tiles = "Stamen Terrain")
weight_pred = [[row['纬度'],row['经度'],row['pred_result']]for index,row in res.iterrows()]
hmap_pred.add_child(HeatMap(weight_pred,gradient={.3:"blue",.6:"green",.9:"yellow",1:"red"}))
hmap_pred